In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader_Lazy, read_and_prepare_data
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvLSTM
from tqdm import tqdm


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
trafo_id = "T1330"
epochs = 25
num_timesteps_in = 12
num_timesteps_out = 4
train_ratio = 0.7
test_ratio_vs_eval_ratio = 0.5
learning_rate = 0.01
device_str = 'cpu'

#----------------------
if device_str == 'mps':
    torch.cuda.empty_cache()

#get dateime string of now
now = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

In [4]:
print("Loading data...")
loader = SimpleGraphVoltDatasetLoader_Lazy(trafo_id, num_timesteps_in, num_timesteps_out)
print(" done")
loader_data_index = loader.snapshot_index

Loading data...
Voltage index: 5
 done


In [7]:
train_dataset, test_eval_dataset = loader.temporal_signal_split_lazy_cut(loader_data_index)
test_dataset, eval_dataset = loader.temporal_signal_split_lazy(test_eval_dataset, test_ratio_vs_eval_ratio)

In [14]:
class TemporalLSTMGNN(torch.nn.Module):
    def __init__(self,node_features, edge_features, periods):
        super(TemporalLSTMGNN, self).__init__()

        out_channels= 32
        K = 5 # size of Chebyshev filter
        self.recurrent_1 = GConvLSTM(
            in_channels=node_features, 
            out_channels=out_channels, 
            K=K, normalization='sym', 
            bias=False)
        
        self.feature_linear = torch.nn.Linear(node_features, node_features)

        self.edge_features_linear = torch.nn.Linear(edge_features, 1)

        self.linear = torch.nn.Linear(out_channels, periods)

    def forward(self, timesteps, edge_index, edge_features):

        #print("edge_features is none: ",torch.isnan(edge_features).any())

        edge_features = self.edge_features_linear(edge_features).squeeze()
        edge_features = F.relu(edge_features)

        #print("edge_features is none: ",torch.isnan(edge_features).any())

        h1, c1 = None, None
        for x in timesteps:

            #print("x is none: ",torch.isnan(x).any())

            x = self.feature_linear(x)

            #print("x is none: ",torch.isnan(x).any())

            h1, c1 = self.recurrent_1(x, edge_index,edge_features,H=h1, C=c1)

            #print("h1 is none: ",torch.isnan(h1).any())
            #print("c1 is none: ",torch.isnan(c1).any())

        x = F.relu(h1)
        x = self.linear(x)

        #print("x is none: ",torch.isnan(x).any())

        return x
    

In [15]:
def train_test(model,device, train_dataset, test_dataset, optimizer, loss_fn, epochs, now):
    """
    Definition of the training loop.
    """
    epoch_losses_train = []
    epoch_losses_test = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss_train = 0

        step = 0
        substeps = 10

        for snapshot_i in tqdm(train_dataset, desc="Training epoch {}".format(epoch)):
            snapshot = loader.get_snapshot(snapshot_i)

            #print(snapshot.x.shape)
            #print(snapshot.edge_index.shape)

            snapshot.to(device)
            optimizer.zero_grad()

            x = snapshot.x.permute(2,0,1)
            #print(x.shape)

            #for x_t in x:
                #print(x_t.shape)

            out = model(x, snapshot.edge_index,snapshot.edge_attr)
            #print(out.shape)

            loss = loss_fn()(out, snapshot.y)
            #print(snapshot.y.shape)
            loss.backward()
            optimizer.step()
            epoch_loss_train += loss.detach().cpu().numpy()

            step += 1

            if step == substeps:
                break

        epoch_losses_train.append(epoch_loss_train)

        #print(f"Epoch {epoch + 1},Weights: {model.state_dict()}")

        model.eval()
        epoch_loss_test = 0
        with torch.no_grad():

            step = 0
            substeps = 10

            for snapshot_j in tqdm(test_dataset, desc="Testing epoch {}".format(epoch)):
                snapshot = loader.get_snapshot(snapshot_j)
                snapshot.to(device)

                x = snapshot.x.permute(2,0,1)
                out = model(x, snapshot.edge_index,snapshot.edge_attr)

                loss = loss_fn()(out, snapshot.y).cpu().numpy()
                epoch_loss_test += loss

                step += 1

                if step == substeps:
                    break

            epoch_losses_test.append(epoch_loss_test)
            if min(epoch_losses_test) == epoch_loss_test:
                torch.save(model.state_dict(), f"../models/A3TGCN_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-{train_ratio}_lr-{learning_rate}.pt")
            print("Epoch: {}, Train Loss: {:.7f}, Test Loss: {:.7f}".format(epoch, epoch_loss_train, epoch_loss_test))
        
        
    return epoch_losses_train, epoch_losses_test

In [16]:
def eval(model, eval_dataset, device, loss_fn, std):
    with torch.no_grad():
        model.eval()
        loss_all = 0
        loss_elementwise = 0
        
        steps = 0
        for snapshot in tqdm(eval_dataset, desc="Evaluating"):
            steps += 1
            snapshot.to(device)
            
            x = snapshot.x.permute(2,0,1)
            out= model(x, snapshot.edge_index,snapshot.edge_attr)

            loss_all += loss_fn()(out, snapshot.y).cpu().numpy()
            loss_elementwise += loss_fn(reduction="none")(out, snapshot.y).cpu().numpy()

            if steps == 10:
                break
            
        loss_all *= std/steps
        loss_elementwise *= std/steps
    return loss_all, loss_elementwise

In [17]:
print("Running training...")
device = torch.device(device_str)
model = TemporalLSTMGNN(node_features=loader.num_features, edge_features=loader.num_edge_features ,periods=num_timesteps_out).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss
losses = train_test(model, device, train_dataset, test_dataset, optimizer, loss_fn, epochs=epochs, now=now)

Running training...


Testing epoch 0:   0%|          | 9/5400 [00:00<09:33,  9.40it/s]


Epoch: 0, Train Loss: 1.2799850, Test Loss: 3.4996683


Testing epoch 1:   0%|          | 9/5400 [00:00<08:56, 10.04it/s]


Epoch: 1, Train Loss: 0.8688666, Test Loss: 3.1366207


Testing epoch 2:   0%|          | 9/5400 [00:00<09:27,  9.50it/s]


Epoch: 2, Train Loss: 0.7946824, Test Loss: 3.2789200


Training epoch 3:   0%|          | 5/10800 [00:00<35:39,  5.05it/s]


KeyboardInterrupt: 

In [24]:
print(losses)

std = loader.mean_and_std["measurements"][1]["voltage"]

#read saved model
model.load_state_dict(torch.load(f"../models/A3TGCN_{now}_{trafo_id}_epochs-{epochs}_in-{num_timesteps_in}_out-{num_timesteps_out}_train-ratio-{train_ratio}_lr-{learning_rate}.pt"))

loss_all, loss_elementwise = eval(model, eval_dataset, device, loss_fn, std)

print("Loss all: {:.7f}".format(loss_all))
print("Loss elementwise: {}".format(loss_elementwise))

NameError: name 'losses' is not defined